# First Data Science Project
## Melbourne Housing Prices Prediction
Here, we will go through a data challenge using data predicting housing prices in Melbourne, Australia. 

The data is from Kaggle and can be found [here](https://www.kaggle.com/anthonypino/melbourne-housing-market)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 1. Collection

In [2]:
df = pd.read_csv("./data/Melbourne_housing_FULL.csv")

## 1.1 Data Assessment

In [3]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
df.shape

(34857, 21)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [6]:
# Take a look to the null values
df.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64

In [7]:
# Number of duplicates
df.duplicated().sum()

1

In [8]:
# get a summary of the categorical variables: number of unique values
cat_var = df.select_dtypes(include=['object']).columns

for i in cat_var:
    print(i, df[i].nunique())

Suburb 351
Address 34009
Type 3
Method 9
SellerG 388
Date 78
CouncilArea 33
Regionname 8


In [9]:
# take a closer look to Suburb
df.Suburb.value_counts()

Reservoir         844
Bentleigh East    583
Richmond          552
Glen Iris         491
Preston           485
                 ... 
Menzies Creek       1
Hopetoun Park       1
Fawkner Lot         1
Avonsleigh          1
croydon             1
Name: Suburb, Length: 351, dtype: int64

In [28]:
# How are the null values of some of the most important variables related
# Rooms and Bedroom2

df[(~df.Bedroom2.isnull()) & (df.Bedroom2 != df.Rooms)][['Bedroom2', 'Rooms']]

,Bedroom2,Rooms
6,3.0,4
7,3.0,4
8,4.0,2
9,3.0,2
11,4.0,3
...,...,...
32335,4.0,3
32768,4.0,3
33976,4.0,5
34451,4.0,3


Over 90% of the data between Bedroom2 and Rooms is the same. Given this I will drop the Bedroom2 Variable as it contains many NaN values

**Data Assessment**
- delete rows with Price, Distance, CouncilArea, Regionname, Propertycount missing values
- get rid of the duplicate
- fill in missing values for the YearBuilt with "NO_INFORMATION"
- change data type to categorical for the following varibales: Postcode, YearBuilt
- remove variables Bedroom2, Longtitude, Lattitude

## 2. Cleaning

In [29]:
# Create a copy of dataframe
df_clean = df.copy()

Delete rows with missing values for the Price, Distance, CouncilArea, Regionname, Propertycount, Bedroom2:

In [30]:
df_clean.dropna(subset=['Price', 'Distance', 'CouncilArea', 'Regionname', 'Propertycount'], 
                inplace=True)

In [31]:
# check deletion works
df.shape[0] - df_clean.shape[0]

7613

In [32]:
# How many NaN are still there
df_clean.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Date                 0
Distance             0
Postcode             0
Bedroom2          6438
Bathroom          6444
Car               6821
Landsize          9262
BuildingArea     16588
YearBuilt        15160
CouncilArea          0
Lattitude         6251
Longtitude        6251
Regionname           0
Propertycount        0
dtype: int64

Drop duplicates

In [33]:
df_clean.drop_duplicates(inplace=True)

In [34]:
# Check for duplicates:
df_clean.duplicated().sum()

0

Change NaN values from YearBuilt with "NO_INFORMARTION". As the missing information could also explain some of the data

In [35]:
df_clean.YearBuilt = df_clean.YearBuilt.replace(np.nan, 'NO_INFORMATION')

In [36]:
# check replacement works
df_clean.YearBuilt.isnull().sum()

0

Change data type to category for Postcode and YearBuilt

In [37]:
# Postcode is an unordered category
df_clean.Postcode = df_clean.Postcode.astype('str')

# YearBuilt is an ordered categorical variable:

# change the category to str so it is compatible with no information 
df_clean.YearBuilt = df_clean.YearBuilt.astype('str')

# create an ordered list of the years:
YearBuilt_order = list(df_clean.YearBuilt.value_counts().sort_index().index)

# create a categorical element:
ordered_YearBuilt = pd.api.types.CategoricalDtype(ordered=True, categories = YearBuilt_order)

# create the category:
df_clean.YearBuilt = df_clean.YearBuilt.astype(ordered_YearBuilt)

In [38]:
# create a function to create ordered categorical values

def create_cat_var(df, var):
    '''
    INPUT:
    - df: DataFrame
    - var: Variable or column to be saved as an ordered category
    
    OUTPUT:
    Dataframe with the correct datatype for the given variable
    '''
    
    # make sure the var is a string
    df[var] = df[var].astype('str')
    
    # create an ordered list sorted by the values
    ordered_list = list(df[var].value_counts().sort_index().index)
    
    # create the categorical element
    cat_element = pd.api.types.CategoricalDtype(ordered=True, categories = ordered_list)
    
    # change the category of the var:
    df[var] = df[var].astype(cat_element)
    
    return df

In [42]:
df_clean.YearBuilt.dtype

CategoricalDtype(categories=['1196.0', '1820.0', '1830.0', '1850.0', '1854.0', '1856.0',
                  '1857.0', '1860.0', '1862.0', '1863.0',
                  ...
                  '2011.0', '2012.0', '2013.0', '2014.0', '2015.0', '2016.0',
                  '2017.0', '2018.0', '2019.0', 'NO_INFORMATION'],
                 ordered=True)

Drop columns: Date, Latitude, Longitude

In [43]:
df_clean.drop(columns = ['Longtitude', 'Lattitude', 'Bedroom2'], inplace = True)

In [44]:
# check the columns were removed
df_clean.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Regionname',
       'Propertycount'],
      dtype='object')

In [45]:
df_clean.shape

(27244, 18)

Save Dataframe as a new csv file

In [46]:
df_clean.to_csv('./data/Melbourne_housing_clean.csv', index=False)